In [3]:
import requests, json, urllib.parse

base_url = 'https://itunes.apple.com/search'

artist = '10AGE'

params = {
    'term': artist,
    'limit': 200,
    'media': 'music',
    'entity': 'song',
    'attribute': 'artistTerm'
}

response = requests.get(f"{base_url}?{urllib.parse.urlencode(params)}")

data = json.loads(response.text)
data = data['results']
import dateutil.parser

# Преобразование строки с датой в объект даты
# **x - скопировать все поля из объекта x
data = map(lambda x: {**x, 'releaseDate': dateutil.parser.isoparse(x['releaseDate'])}, data)
data = list(data)
# Убираем лишние поля
data = map(lambda x: {
    'artistId': x['artistId'],
    'collectionId': x['collectionId'],
    'trackId': x['trackId'],
    'artistName': x['artistName'],
    'collectionName': x['collectionCensoredName'],
    'trackName': x['trackCensoredName'],
    'collectionPrice': x['collectionPrice'],
    'trackPrice': x['trackPrice'],
    'releaseDate': x['releaseDate'],
    'collectionExplicitness': x['collectionExplicitness'],
    'trackExplicitness': x['trackExplicitness'],
    'discCount': x['discCount'],
    'discNumber': x['discNumber'],
    'trackCount': x['trackCount'],
    'trackNumber': x['trackNumber'],
    'trackTimeMillis': x['trackTimeMillis'],
    'country': x['country'],
    'currency': x['currency'],
    'primaryGenreName': x['primaryGenreName'],
    'isStreamable': x['isStreamable']
}, data)
data = list(data)

# Фильтрация с целью оставить искомого артиста
data = filter(lambda x: artist.lower() in x['artistName'].lower(), data)
data = list(data)
# Функция переводящая миллисекунды в читаемый формат времени
def convert_millis(millis: int) -> str:
    total_seconds = millis // 1000
    seconds = total_seconds % 60
    minutes = (total_seconds // 60) % 60
    hours = (total_seconds // (60 * 60))
    return f"{hours}:{minutes}:{seconds}"
from statistics import mean, median
result = {}
result['Artist Name'] = data[0]['artistName']
result['Track Amount'] = len(set(list(map(lambda x: x['trackName'], data))))
result['Popular Genre'] = data[0]['primaryGenreName']
result['Album Amount'] = len(set(list(map(lambda x: x['collectionId'], data))))

result['Total song time'] = convert_millis(sum(set(list(map(lambda x: x['trackTimeMillis'], data)))))
result['Mean song time'] = convert_millis(mean(set(list(map(lambda x: x['trackTimeMillis'], data)))))
result['Median song time'] = convert_millis(median(sorted(set(list(map(lambda x: x['trackTimeMillis'], data))))))
result['Longest name'] = max(data, key=lambda x: x['trackTimeMillis'])['trackName']
result['Longest time'] = convert_millis(max(data, key=lambda x: x['trackTimeMillis'])['trackTimeMillis'])
result['Shortest name'] = min(data, key=lambda x: x['trackTimeMillis'])['trackName']
result['Shortest time'] = convert_millis(min(data, key=lambda x: x['trackTimeMillis'])['trackTimeMillis'])

result['Total song price'] = sum(list(map(lambda x: x['trackPrice'], data)))
result['Mean song price'] = mean((list(map(lambda x: x['trackPrice'], data))))
result['Median song price'] = median(sorted((list(map(lambda x: x['trackPrice'], data)))))
result['Expensive song name'] = max(data, key=lambda x: x['trackPrice'])['trackName']
result['Most expensive song price'] = max(data, key=lambda x: x['trackPrice'])['trackPrice']
result['Cheapest song name'] = min(data, key=lambda x: x['trackPrice'])['trackName']
result['Cheapest song price'] = min(data, key=lambda x: x['trackPrice'])['trackPrice']

for key, value in result.items():
    print(key + ': ', value)

from itertools import groupby

album_name = set(list(map(lambda x: x['collectionName'], data)))
album_id = set(list(map(lambda x: x['collectionId'], data)))

print(album_name)
print(album_id)

collection_id_extractor = lambda x: x['collectionId']

sorted_by_album = sorted(data, key=collection_id_extractor)
album_groups = {}
album_names = {}

for album_id, songs in groupby(sorted_by_album, key=collection_id_extractor):
    songs = list(songs)
    album_groups[album_id] = songs 
    album_names[album_id] = songs[0]['collectionName']
del sorted_by_album

print(list(album_names.values()))
from statistics import mean, median
result2 = {}

album = album_groups[1504918800]

result2['Album Name'] = album_names[1504918800]
result2['Track Amount'] = len(set(list(map(lambda x: x['trackName'], album))))
result2['Primary Genre'] = list(map(lambda x: x['primaryGenreName'], album))[0]
result2['Album Release'] = album[0]['releaseDate']

result2['Total song time'] = convert_millis(sum(set(list(map(lambda x: x['trackTimeMillis'], album)))))
result2['Mean song time'] = convert_millis(mean(set(list(map(lambda x: x['trackTimeMillis'], album)))))
result2['Median song time'] = convert_millis(int(median(sorted(set(list(map(lambda x: x['trackTimeMillis'], album)))))))
result2['Longest name'] = max(album, key=lambda x: x['trackTimeMillis'])['trackName']
result2['Longest time'] = convert_millis(max(album, key=lambda x: x['trackTimeMillis'])['trackTimeMillis'])
result2['Shortest name'] = min(album, key=lambda x: x['trackTimeMillis'])['trackName']
result2['Shortest time'] = convert_millis(min(album, key=lambda x: x['trackTimeMillis'])['trackTimeMillis'])

result2['Total song price'] = sum(list(map(lambda x: x['trackPrice'], album)))
result2['Mean song price'] = mean((list(map(lambda x: x['trackPrice'], album))))
result2['Median song price'] = median(sorted((list(map(lambda x: x['trackPrice'], album)))))
result2['Expensive song name'] = max(album, key=lambda x: x['trackPrice'])['trackName']
result2['Most expensive song price'] = max(album, key=lambda x: x['trackPrice'])['trackPrice']
result2['Cheapest song name'] = min(album, key=lambda x: x['trackPrice'])['trackName']
result2['Cheapest song price'] = min(album, key=lambda x: x['trackPrice'])['trackPrice']

for key, value in result2.items():
    print(key + ': ', value)

Artist Name:  10AGE
Track Amount:  28
Popular Genre:  Hip-Hop
Album Amount:  26
Total song time:  1:25:32
Mean song time:  0.0:3.0:3.0
Median song time:  0.0:3.0:0.0
Longest name:  Туман
Longest time:  0:4:8
Shortest name:  ОТЦАСИТИ
Shortest time:  0:1:45
Total song price:  34.79999999999998
Mean song price:  1.16
Median song price:  1.29
Expensive song name:  Ау
Most expensive song price:  1.29
Cheapest song name:  Пушка
Cheapest song price:  0.99
{'Время - Single', 'ОТЦАСИТИ - Single', 'Пушка - Single', 'Лия - Single', 'До луны - Single', 'Я тот, кто довезёт - Single', 'Милая - Single', 'Питерское такси - Single', 'Зоопарк - Single', 'Паровозик - Single', 'Raspberry - Single', 'День21 - Single', 'Туман - Single', 'Ау - EP', 'Ау - Single', 'ОПДАВАЙДАВАЙ - Single', 'Зацелую - Single', 'Моя Suicide - Single', 'Нету интереса - Single', 'Пишешь мне пока - Single', 'Малышка - Single', 'Мандариновые горы - Single', 'На колени! - Single', 'Камеры врут - Single', 'Али ты - Single', 'CD-ROM - 